In [45]:
import pandas as pd
import numpy as np
import math

In [2]:
articles = pd.read_csv('../data/articles/shared_articles.csv')
interactions = pd.read_csv('../data/articles/users_interactions.csv')

In [4]:
articles.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [8]:
articles = articles[articles['eventType'] == 'CONTENT SHARED']
articles.shape

(3047, 13)

In [10]:
interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [13]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions['eventType'] = interactions['eventType'].map(event_type)

In [16]:
mean_value = interactions['eventType'].mean()
print(f"Mean - {mean_value:.2f}")

Mean - 1.24


In [43]:
users_df = interactions.groupby(['personId', 'contentId']).count().reset_index().groupby('personId').count()['contentId']
users_5_df = users_df[users_df >= 5].reset_index()['personId']
print(len(users_5_df))

1140


In [44]:
int_from_users5_df = interactions.loc[np.in1d(interactions.personId, users_5_df)] 
print(int_from_users5_df.shape)

(69868, 8)


In [46]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)

In [62]:
full_df = int_from_users5_df.groupby(['personId', 'contentId'])['eventType'].sum().apply(smooth_user_preference).reset_index().set_index(['personId', 'contentId'])
full_df['last_timestamp'] = int_from_users5_df.groupby(['personId', 'contentId'])['timestamp'].max()
full_df['last_timestamp'].mean()

1470605340.0403006

In [64]:
train_df = full_df.loc[full_df.last_timestamp < 1475519545].copy()
test_df = full_df.loc[full_df.last_timestamp >= 1475519545].copy()
 
print(len(train_df))

29325


In [72]:
popular =  train_df.groupby('contentId')['eventType'].sum().reset_index().sort_values('eventType', ascending=False)['contentId'].values

popular[0]

-6783772548752091658

In [73]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

NameError: name 'final_df' is not defined